# Setup

In [4]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [5]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback

In [6]:
import config
#Erlend
#wandb.login(key=config.erlend_key)
# Hjalmar
wandb.login(key=config.hjalmar_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vinje (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hjalmarjacobvinje/.netrc


True

# Load, split and normalize data

### Load data

In [7]:
file = "../data/processed_data/2022.csv"
df_read = pd.read_csv(file)
display(df_read)

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,...,Underlying_82,Underlying_83,Underlying_84,Underlying_85,Underlying_86,Underlying_87,Underlying_88,Underlying_89,Underlying_90,R
0,721776,2022-05-10,2022-05-11,3002.45,4000.99,1000.0,1,4.000990,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
1,721777,2022-05-10,2022-05-11,2800.60,4000.99,1200.0,1,3.334158,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
2,721778,2022-05-10,2022-05-11,2602.20,4000.99,1400.0,1,2.857850,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
3,721779,2022-05-10,2022-05-11,2402.45,4000.99,1600.0,1,2.500619,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
4,721780,2022-05-10,2022-05-11,2202.45,4000.99,1800.0,1,2.222772,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
5,721781,2022-05-10,2022-05-11,1998.15,4000.99,2000.0,1,2.000495,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
6,721782,2022-05-10,2022-05-11,1799.25,4000.99,2200.0,1,1.818632,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
7,721783,2022-05-10,2022-05-11,1601.70,4000.99,2400.0,1,1.667079,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
8,721784,2022-05-10,2022-05-11,1402.25,4000.99,2600.0,1,1.538842,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57
9,721785,2022-05-10,2022-05-11,1202.45,4000.99,2800.0,1,1.428925,7.73,-130.56,...,1.57,-65.91,13.02,43.68,-6.56,-19.84,-4.39,-92.55,-2.38,0.57


### Format input data

In [8]:
df = df_read

# Formating settings
val_start = "2022-11-01"
num_features = 4
time_steps = 20
bs_vars = ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, time_steps)]

# Split train and validation data
df_train = df[df['Quote_date'] < val_start]
df_val = df[df['Quote_date'] >= val_start]

# Extract target values
train_y = df_train['Price'].to_numpy()
val_y = df_val['Price'].to_numpy()

# Convert dataframes to numpy arrays
train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0])
val_x[0] = underlying_scaler.transform(val_x[0])

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]),time_steps,1), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), time_steps, 1), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

Train shape: (918922, 20, 1), (918922, 4)
Val shape: (311065, 20, 1), (311065, 4)


# Model construction

In [9]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [14]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 testing : 21-22 data',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [8, 16, 32, 64, 96, 128]},
        'MLP_units': {
            'values': [32, 64, 96, 128]},
        'LSTM_timesteps': {
            'values': [20]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 2},
        'Bn_momentum': {
            'distribution': 'uniform',
            'max': 1, 'min': 0},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.01, 'min': 0.0001},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.9},        
        'Minibatch_size': {
            'values': [1024, 2048, 4096]},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

Create sweep with ID: ybzdubg7
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/ybzdubg7


# Run hyperparameter search

In [15]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        # Adapt sequence length to config
        train_x[0] = train_x[0][:, :config.LSTM_timesteps, :]
        val_x[0] = val_x[0][:, :config.LSTM_timesteps, :]
        print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
        print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

        # Train model
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback] 
        )

### Run full sweep

In [16]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 1)

wandb: Agent Starting Run: x1nwz702 with config:
wandb: 	Bn_momentum: 0.2115311974369981
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 64
wandb: 	Lr: 0.007252310655150062
wandb: 	Lr_decay: 0.931385828545152
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 128
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 1024
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 64)           115968      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 68)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

wandb: Network error (ConnectionError), entering retry loop.


898/898 [==============================] - 3373s 4s/step - loss: 1178.8127 - val_loss: 2081.3750
Epoch 16/1000
898/898 [==============================] - 470s 524ms/step - loss: 1030.3053 - val_loss: 4746.7500
Epoch 17/1000
414/898 [============>.................] - ETA: 9:46 - loss: 1035.6847

wandb: Network error (ConnectionError), entering retry loop.


...

wandb: Network error (ConnectionError), entering retry loop.


685/898 [=====================>........] - ETA: 4:37 - loss: 1001.6516

wandb: Network error (ConnectionError), entering retry loop.


898/898 [==============================] - 1393s 2s/step - loss: 1035.3822 - val_loss: 1996.6251
Epoch 18/1000
898/898 [==============================] - 433s 481ms/step - loss: 977.9628 - val_loss: 918.9311
Epoch 19/1000
147/898 [===>..........................] - ETA: 4:55 - loss: 965.1594

wandb: Ctrl + C detected. Stopping sweep.


### Single run

In [ ]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 1,
    'Patience': 20,
    'Num_features': 4,
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': '2022: Val split 01.11 with shuffle',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 96)           111744      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 100)          0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb